In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import os


print("show data file")

for f in os.listdir("../input"):
    if 'zip' not in f:
        print(f.ljust(30) + str(round(os.path.getsize('../input/' + f) / 1000000, 2)) + 'MB')
        
df_train = pd.read_csv('../input/train.csv', nrows=100000)
df_test = pd.read_csv('../input/test.csv', nrows=100000)

print("Head of train:")
df_train.head(20)


In [ ]:

print("Head of test:")
df_test.head(20)



In [ ]:
print(df_train.nunique())
df_train.describe()

In [ ]:
print(df_test.nunique())
df_test.describe()

train和test数据unique之后，分析数据发现一个比较奇怪的现象，click_time相对还是比较集中的，并不像ip那样分布很广

下面分析几个集中的数据（app,device,os,channel,click_time）

In [ ]:
app = df_train.app.value_counts(normalize=True)
device = df_train.device.value_counts(normalize=True)
os = df_train.os.value_counts(normalize=True)

plt.figure(figsize=(12,15))

plt.subplot(311)
g1 = sns.barplot(x=app.index[:20], y =app.values[:20])
g1.set_title("App Dist", fontsize=15)
g1.set_xlabel("App ID")
g1.set_ylabel("count normalize", fontsize=12)

plt.subplot(312)
g2 = sns.barplot(x=device.index[:20], y=device.values[:20])
g2.set_title("Device Dist", fontsize=15)
g2.set_xlabel("Device ID")
g2.set_ylabel("count normalize", fontsize=12)

plt.subplot(313)
g3 = sns.barplot(x=os.index[:20], y=os.values[:20])
g3.set_title("OS Dist", fontsize=15)
g3.set_xlabel("OS ID")
g3.set_ylabel("count normalize", fontsize=12)

plt.subplots_adjust(hspace=0.3)
plt.show()

print(app.loc[app.values > 0.05])
print(device[:3])
print(os[:5])

分析app可以看到占比大于0.05的app只有(9,12,15,8,3,2,18,1)，相对并不是特别集中
分析device，可以明显看到94.4%的device集中在id 1上，非常集中
分析os，os也相对比较集中在id 19和13上
这三维数据都是id与实物对应，理论上应该相对比较集中的。实际数据展示，app集中度一般，device特别集中（不应该呀，与预期不一样...前10W行数据的原因？）

In [ ]:
channel = df_train.channel.value_counts(normalize=True)
click_t = df_train.click_time.value_counts(normalize=True)

plt.figure(figsize=(12,10))

plt.subplot(211)
g1 = sns.barplot(x=channel.index[:20], y=channel.values[:20])
g1.set_title("channel dist", fontsize=15)
g1.set_xlabel("channel id")
g1.set_ylabel("count normalize")

plt.subplot(212)
g2 = sns.barplot(x=click_t.index[:20], y=click_t.values[:20])
g2.set_xticklabels(g2.get_xticklabels(), rotation=90)
g2.set_title("click time dist", fontsize=15)
g2.set_xlabel("click time")
g2.set_ylabel("count normalize")

plt.subplots_adjust(hspace=0.3)
plt.show()

channel id与app类似，主要集中在134,145,245，但是集中度一般

click_time的分布相对均匀，比较奇怪的是click_time的所有时间点比想象中的少很多（前10W数据的原因？）

接下来，我们把click_time拆分后看一下分布

In [ ]:
df_train['click_time'] = pd.to_datetime(df_train['click_time'])

df_train['click_time_D'] = df_train['click_time'].dt.to_period("H")
print("click_time limit to hour: ")
print(df_train['click_time_D'].unique())
#print(df_train['click_time_D'][:10])
df_train['click_time_m'] = df_train['click_time'].dt.to_period("min")
# print(df_train['click_time_m'][:10])
print("click_time limit to min, size: ", len(df_train['click_time_m'].unique()))

df_test['click_time'] = pd.to_datetime(df_test['click_time'])

df_test['click_time_D'] = df_test['click_time'].dt.to_period("H")
print("click_time limit to hour: ")
print(df_test['click_time_D'].unique())
#print(df_train['click_time_D'][:10])
df_test['click_time_m'] = df_test['click_time'].dt.to_period("min")
# print(df_train['click_time_m'][:10])
print("click_time limit to min, size: ", len(df_test['click_time_m'].unique()))

可以看到，把click_time限制到小时级，只有三个时间点（why？），限制到分钟级，也只有75个

而测试集则是另外一个时间点

猜测，这里训练集和测试机都只是统计了部分时段的数据，所以click_time没有参考价值

接下来分析一下，在训练集中app被下载时，各维度数据的分布

In [ ]:
dw = df_train.loc[df_train['is_attributed'] == 1]
app_dw = dw.app.value_counts(normalize=True)
device_dw = dw.device.value_counts(normalize=True)
os_dw = dw.os.value_counts(normalize=True)

plt.figure(figsize=(12,15))

plt.subplot(311)
g1 = sns.barplot(x=app_dw.index[:20], y =app_dw.values[:20])
g1.set_title("App Dist", fontsize=15)
g1.set_xlabel("App ID")
g1.set_ylabel("count normalize", fontsize=12)

plt.subplot(312)
g2 = sns.barplot(x=device_dw.index[:20], y=device_dw.values[:20])
g2.set_title("Device Dist", fontsize=15)
g2.set_xlabel("Device ID")
g2.set_ylabel("count normalize", fontsize=12)

plt.subplot(313)
g3 = sns.barplot(x=os_dw.index[:20], y=os_dw.values[:20])
g3.set_title("OS Dist", fontsize=15)
g3.set_xlabel("OS ID")
g3.set_ylabel("count normalize", fontsize=12)

plt.subplots_adjust(hspace=0.3)
plt.show()

print(app[:3])
print(app_dw[:3])
print(device[:3])
print(device_dw[:3])
print(os[:3])
print(os_dw[:3])

对比数据可以明显发现app对是否下载的贡献度很高，其他维度分布基本一致

In [1]:
plt.figure(figsize=(12,20))
plt.subplot(411)
ax1 = sns.distplot(df_train['app'][df_train['is_attributed']==0], color='r', label='no download')
ax2 = sns.distplot(df_train['app'][df_train['is_attributed']==1], color='b', label='download')

plt.subplot(412)
ax1 = sns.distplot(df_train['device'][df_train['is_attributed']==0], color='r', label='no download')
ax2 = sns.distplot(df_train['device'][df_train['is_attributed']==1], color='b', label='download')

plt.subplot(413)
ax1 = sns.distplot(df_train['os'][df_train['is_attributed']==0], color='r', label='no download')
ax2 = sns.distplot(df_train['os'][df_train['is_attributed']==1], color='b', label='download')

plt.subplot(414)
ax1 = sns.distplot(df_train['channel'][df_train['is_attributed']==0], color='r', label='no download')
ax2 = sns.distplot(df_train['channel'][df_train['is_attributed']==1], color='b', label='download')
plt.show()

我们画出这几个主要信号，在app是否被下载的情况下的分布图，可以看到os，channel的分布差别不大，而且app下载量和投放量有一定正相关，但是有个别app下载率很高

In [2]:
通过分析可以发现可用的5维数据中，click_time由于数据原因基本是没有贡献度的，剩下的几维数据中，app和device和channel会是相对比较有贡献度的

而ip由于经过了编码，不能做分段处理，贡献度应该也很低。

基本上是一个很少特征的二分类问题，直观上使用xgboost处理。